# 使用自编码器提取特征


## 1. 读取筛选后的数据


In [1]:
import pandas as pd
import torch

# 选取需要的列
COLUMN_INDEX = [
    "Index",
    "Temp (°C)",
    "Dew Point Temp (°C)",
    "Rel Hum (%)",
    "Wind Spd (km/h)",
    "Stn Press (kPa)",
]
data = pd.read_csv("Data/DataSet.csv")[COLUMN_INDEX].to_numpy()

data_tensor = torch.tensor(data, dtype=torch.float32)
data_tensor

tensor([[ 0.0000e+00,  1.5000e+00, -3.6000e+00,  6.9000e+01,  3.9000e+01,
          9.9810e+01],
        [ 1.0000e+00,  1.5000e+00, -3.9000e+00,  6.7000e+01,  3.5000e+01,
          1.0001e+02],
        [ 2.0000e+00,  1.0000e+00, -4.3000e+00,  6.8000e+01,  3.2000e+01,
          1.0014e+02],
        ...,
        [ 1.7517e+04,  4.0000e+00,  3.6000e+00,  9.7000e+01,  3.2000e+01,
          9.9150e+01],
        [ 1.7518e+04,  4.0000e+00,  3.6000e+00,  9.7000e+01,  3.0000e+01,
          9.8800e+01],
        [ 1.7519e+04,  4.0000e+00,  3.7000e+00,  9.8000e+01,  1.5000e+01,
          9.8570e+01]])

## 2. 模型训练


划分训练集和测试集


In [2]:
from Network import *

# 设置随机种子以确保结果可重复
torch.manual_seed(42)

train_loader,test_loader,train_index,test_index=TimeSeriesDataSplit2Loaders(
    data_tensor,
    BATCH_SIZE=16,
    RATIO=0.5,SHUFFLE=True)


定义eDMD损失函数

$$
\mathbf{Loss}_{\text{eDMD}} = n_{1} \cdot \mathbf{Loss}_{\text{MSE}} + n_{2} \cdot \mathbf{Loss}_{\text{Measurement Space Linearity}}.
$$


定义Auto Encoder结构


In [3]:
from torchinfo import summary

MyAE = MyAutoencoder(input_size=5, hidden_size_1=6,
                     hidden_size_2=3, dropout_prob=0.05)
INIT_WEIGHTS_XAVIER(MyAE)
summary(MyAE, (1, 5))

Layer (type:depth-idx)                   Output Shape              Param #
MyAutoencoder                            [1, 5]                    --
├─Sequential: 1-1                        [1, 3]                    --
│    └─Linear: 2-1                       [1, 6]                    36
│    └─ReLU: 2-2                         [1, 6]                    --
│    └─BatchNorm1d: 2-3                  [1, 6]                    12
│    └─Dropout: 2-4                      [1, 6]                    --
│    └─Linear: 2-5                       [1, 3]                    21
│    └─ReLU: 2-6                         [1, 3]                    --
│    └─BatchNorm1d: 2-7                  [1, 3]                    6
│    └─Dropout: 2-8                      [1, 3]                    --
├─Sequential: 1-2                        [1, 5]                    --
│    └─Linear: 2-9                       [1, 6]                    24
│    └─ReLU: 2-10                        [1, 6]                    --
│    └─BatchNorm

训练模型


In [4]:
optimizer = torch.optim.Adam(MyAE.parameters(), lr=0.01, weight_decay=1E-5)
TRAIN_WITH_PROGRESS_BAR_eDMD(
    MyAE,
    NUM_EPOCHS=2,
    OPTIMIZER=optimizer,
    TRAIN_LOADER=train_loader,
    TEST_LOADER=test_loader,
    TORCH_LOSS_TYPE=nn.MSELoss(),
    LOSS_WEIGHT=[0.5,0.5],
    DEVICE=0,
)

PyTorch Version: 2.1.0.dev20230621+cu117
Training on cuda:0
====================================Start training====================================


Epoch 2/2: 100%|██████████| 548/548 [00:01<00:00, 328.05batch/s, loss=66.1]


====================================Finish training====================================



([tensor(557.1006), tensor(37.5786)], [tensor(24.2231), tensor(15.0917)])